# HO01: Similaridade Textual
**Vencimento** Terça-feira por 7:00 <br>
**Pontos** 5 <br>
Esta é a tarefa **HO01: Similaridade Textual**, uma atividade prática que estimula o aluno a desenvolver habilidades de programação em linguagem Python para processamento textual.

## Problema
Calcular a similaridade entre cada par de documentos no coleção headlines.txt (um documento por linha):

1. Pré-processar cada documento para tokenizar, remover acentos e caracteres especiais, fazer lematização e stemming;
2. Criar a representação vetorial de 5 formas diferentes: Onehot Encoding, Counting Vectors, TF-IDF, Co-occurrence Vectors, Word2Vec;
3. Calcular a similaridade par-a-par usando 2 formas diferentes: Euclidean, Cosine

## Pré-requisitos
- pip install nltk
- pip install sklearn
- pip install gensim

In [151]:
# Bibliotecas necessárias, funções que só precisam ser executadas uma vez e carregar o arquivo de texto
import re
import nltk
from nltk.tokenize import word_tokenize
import unicodedata
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer

import pandas as pd
from IPython.display import display
from colorama import Fore, Back, Style       #prints coloridos e estilizados no terminal
# Bibliotecas da parte 2
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import Binarizer
from gensim.models import Word2Vec


# Baixar os recursos do NLTK necessários (apenas uma vez)
nltk.download('punkt')
nltk.download('wordnet')

def print_destaque(texto):
    print(Back.LIGHTYELLOW_EX + Fore.BLUE + Style.BRIGHT + f' {texto} '+ Style.RESET_ALL)

texto_com_caracteres_especiais = "Olá, Coração? Café & Música são R$5,00. Avôs são fáceis de tê-los. Orações no sótão grátis! 😃"

# Carregar o arquivo de texto
with open('headlines.txt', 'r') as f:  
    headlines = f.readlines()

print_destaque("Coleção de Documentos (headlines.txt)")
headlines

 Coleção de Documentos (headlines.txt) 


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rodri\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\rodri\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


['Investors unfazed by correction as crypto funds see $154 million inflows\n',
 'Bitcoin, Ethereum prices continue descent, but crypto funds see inflows\n',
 'The surge in euro area inflation during the pandemic: transitory but with upside risks\n',
 "Inflation: why it's temporary and raising interest rates will do more harm than good\n",
 'Will Cryptocurrency Protect Against Inflation?\n',
 'Tweed is a crypto wallet API to add a web3 flavor to any web service\n',
 'Who Created Bitcoin? Learn About The Biggest Cryptos, Including Dogecoin, Big Eyes Coin\n',
 'Cryptocurrency Prices And News: Bitcoin, Cryptos Fall After Silvergate Bank Liquidation News\n',
 'Silvergate Capital To Shut Down, Liquidate Crypto-Friendly Silvergate Bank\n',
 'Dow Jones Rises On Surprise Jump In Jobless Claims; Bitcoin Drops As Crypto Bank Silvergate Crashes 40%\n']

## Pré-processar cada documento para tokenizar, remover acentos e caracteres especiais, fazer lematização e stemming

### Tokenização
É o processo de dividir um texto em unidades menores, chamadas de tokens. Os tokens podem ser palavras, frases, sentenças ou até mesmo caracteres individuais, dependendo do nível de granularidade desejado. A tokenização é uma etapa fundamental em muitas tarefas de processamento de linguagem natural (NLP) porque permite que o texto seja processado em unidades significativas.

In [104]:
# Tokenização de palavras
def tokenizar(texto):
    return word_tokenize(texto) 

print_destaque("Tokenização de palavras")
print(*[tokenizar(texto) for texto in headlines], sep='\n')

 Tokenização de palavras 
['Investors', 'unfazed', 'by', 'correction', 'as', 'crypto', 'funds', 'see', '$', '154', 'million', 'inflows']
['Bitcoin', ',', 'Ethereum', 'prices', 'continue', 'descent', ',', 'but', 'crypto', 'funds', 'see', 'inflows']
['The', 'surge', 'in', 'euro', 'area', 'inflation', 'during', 'the', 'pandemic', ':', 'transitory', 'but', 'with', 'upside', 'risks']
['Inflation', ':', 'why', 'it', "'s", 'temporary', 'and', 'raising', 'interest', 'rates', 'will', 'do', 'more', 'harm', 'than', 'good']
['Will', 'Cryptocurrency', 'Protect', 'Against', 'Inflation', '?']
['Tweed', 'is', 'a', 'crypto', 'wallet', 'API', 'to', 'add', 'a', 'web3', 'flavor', 'to', 'any', 'web', 'service']
['Who', 'Created', 'Bitcoin', '?', 'Learn', 'About', 'The', 'Biggest', 'Cryptos', ',', 'Including', 'Dogecoin', ',', 'Big', 'Eyes', 'Coin']
['Cryptocurrency', 'Prices', 'And', 'News', ':', 'Bitcoin', ',', 'Cryptos', 'Fall', 'After', 'Silvergate', 'Bank', 'Liquidation', 'News']
['Silvergate', 'Capita

### Normalização
É o processo de transformar o texto em uma forma canônica, eliminando variações desnecessárias, como remoção de acentos, caracteres especiais, letras maiúsculas/minúsculas, e outras formas de padronização. Isso ajuda a reduzir a dimensionalidade dos dados e a simplificar o texto para análise posterior.
1. Remover acentos
2. Remover caracteres especiais
3. Colocar todas em minúscula

In [95]:
def remover_acentos(texto):
    return unicodedata.normalize('NFKD', texto).encode('ASCII', 'ignore').decode('ASCII') # Remove acentos
    # variação ao código acima: ''.join(c for c in unicodedata.normalize('NFD', texto) if unicodedata.category(c) != 'Mn')

def remover_caracteres_especiais(texto):
    texto = remover_acentos(texto)
    return re.sub(r'[^a-zA-Z0-9\s]', '', texto) # Remove caracteres especiais

def normalizar(texto):
    return remover_caracteres_especiais(texto).lower() 

print('ORIGINAL:       ', texto_com_caracteres_especiais)
print('SEM ACENTOS:    ', remover_acentos(texto_com_caracteres_especiais))
print('+ S/ CARAC.ESP: ', remover_caracteres_especiais(texto_com_caracteres_especiais))
print('+ CAIXA BAIXA:  ', normalizar(texto_com_caracteres_especiais), '\n')

print_destaque("Normalização dos Textos")
print(*[normalizar(texto) for texto in headlines], sep='')   

ORIGINAL:        Olá, Coração? Café & Música são R$5,00. Avôs são fáceis de tê-los. Orações no sótão grátis! 😃
SEM ACENTOS:     Ola, Coracao? Cafe & Musica sao R$5,00. Avos sao faceis de te-los. Oracoes no sotao gratis! 
+ S/ CARAC.ESP:  Ola Coracao Cafe  Musica sao R500 Avos sao faceis de telos Oracoes no sotao gratis 
+ CAIXA BAIXA:   ola coracao cafe  musica sao r500 avos sao faceis de telos oracoes no sotao gratis  

 Normalização dos Textos 
investors unfazed by correction as crypto funds see 154 million inflows
bitcoin ethereum prices continue descent but crypto funds see inflows
the surge in euro area inflation during the pandemic transitory but with upside risks
inflation why its temporary and raising interest rates will do more harm than good
will cryptocurrency protect against inflation
tweed is a crypto wallet api to add a web3 flavor to any web service
who created bitcoin learn about the biggest cryptos including dogecoin big eyes coin
cryptocurrency prices and news bitcoin

### Lematização
É o processo de reduzir palavras a sua forma base ou raiz, conhecida como lemma. A lematização considera a estrutura morfológica das palavras e pode ser útil para agrupar palavras derivadas da mesma raiz em um único token, reduzindo a redundância de informações.

In [112]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

#palavras = ["correndo", "corre", "correu", "corridas"]
#lemmas = [lemmatizer.lemmatize(palavra) for palavra in palavras]
#print(lemmas)

def lematizar_doc(texto):
    return [lemmatizer.lemmatize(palavra) for palavra in tokenizar(normalizar(texto))]

print_destaque("Lematização dos Textos")
print(*[lematizar_doc(texto) for texto in headlines], sep='\n')

 Lematização dos Textos 
['investor', 'unfazed', 'by', 'correction', 'a', 'crypto', 'fund', 'see', '154', 'million', 'inflow']
['bitcoin', 'ethereum', 'price', 'continue', 'descent', 'but', 'crypto', 'fund', 'see', 'inflow']
['the', 'surge', 'in', 'euro', 'area', 'inflation', 'during', 'the', 'pandemic', 'transitory', 'but', 'with', 'upside', 'risk']
['inflation', 'why', 'it', 'temporary', 'and', 'raising', 'interest', 'rate', 'will', 'do', 'more', 'harm', 'than', 'good']
['will', 'cryptocurrency', 'protect', 'against', 'inflation']
['tweed', 'is', 'a', 'crypto', 'wallet', 'api', 'to', 'add', 'a', 'web3', 'flavor', 'to', 'any', 'web', 'service']
['who', 'created', 'bitcoin', 'learn', 'about', 'the', 'biggest', 'cryptos', 'including', 'dogecoin', 'big', 'eye', 'coin']
['cryptocurrency', 'price', 'and', 'news', 'bitcoin', 'cryptos', 'fall', 'after', 'silvergate', 'bank', 'liquidation', 'news']
['silvergate', 'capital', 'to', 'shut', 'down', 'liquidate', 'cryptofriendly', 'silvergate', 'b

### Stemming 
É o processo de reduzir palavras à sua forma radical ou base, conhecida como stem. Ao contrário da lematização, o stemming não leva em consideração a estrutura morfológica das palavras, apenas remove os sufixos para obter a forma básica da palavra. Isso pode resultar em palavras que não são reconhecíveis em seu sentido original, mas pode ser útil em certos cenários onde a redução de palavras a sua forma mais básica é desejada.

In [125]:
from nltk.stem import PorterStemmer
#from nltk.stem import SnowballStemmer
#stemmer = SnowballStemmer("portuguese") # Escolha o idioma desejado
#stemmer.stem("correndo")

stemmer = PorterStemmer()
def stemming_doc(texto):
    return [stemmer.stem(palavra) for palavra in tokenizar(normalizar(texto))]

print_destaque("Stemming dos Textos")
print(*[stemming_doc(texto) for texto in headlines], sep='\n')

 Stemming dos Textos 
['investor', 'unfaz', 'by', 'correct', 'as', 'crypto', 'fund', 'see', '154', 'million', 'inflow']
['bitcoin', 'ethereum', 'price', 'continu', 'descent', 'but', 'crypto', 'fund', 'see', 'inflow']
['the', 'surg', 'in', 'euro', 'area', 'inflat', 'dure', 'the', 'pandem', 'transitori', 'but', 'with', 'upsid', 'risk']
['inflat', 'whi', 'it', 'temporari', 'and', 'rais', 'interest', 'rate', 'will', 'do', 'more', 'harm', 'than', 'good']
['will', 'cryptocurr', 'protect', 'against', 'inflat']
['tweed', 'is', 'a', 'crypto', 'wallet', 'api', 'to', 'add', 'a', 'web3', 'flavor', 'to', 'ani', 'web', 'servic']
['who', 'creat', 'bitcoin', 'learn', 'about', 'the', 'biggest', 'crypto', 'includ', 'dogecoin', 'big', 'eye', 'coin']
['cryptocurr', 'price', 'and', 'news', 'bitcoin', 'crypto', 'fall', 'after', 'silverg', 'bank', 'liquid', 'news']
['silverg', 'capit', 'to', 'shut', 'down', 'liquid', 'cryptofriendli', 'silverg', 'bank']
['dow', 'jone', 'rise', 'on', 'surpris', 'jump', 'in', 

## Criar a representação vetorial de 5 formas diferentes: Onehot Encoding, Counting Vectors, TF-IDF, Co-occurrence Vectors, Word2Vec

### One-Hot Encoding
É uma técnica de representação vetorial em que cada palavra é representada como um vetor binário com um valor "1" na posição correspondente à palavra e "0" em todas as outras posições. É comumente usada para representar categorias discretas ou palavras em um vocabulário limitado.

In [164]:
def onehot_encode_doc(texto):    
    encoder = OneHotEncoder() # Criando uma instância do OneHotEncoder    
    text_encoded = encoder.fit_transform([[palavra] for palavra in tokenizar(normalizar(texto))]) # Codificando o texto usando o OneHotEncoder
    
    pd.set_option('display.precision', 0) # configurando para não exibir casas decimais
    df_encoded = pd.DataFrame(text_encoded.toarray(), columns=encoder.get_feature_names_out([''])) # Criando um DataFrame com os dados codificados

    print("Matriz One-Hot Encoding de: ", texto[:-1])
    display(df_encoded)

print_destaque("One-Hot Encoding dos Textos")
[onehot_encode_doc(texto) for texto in headlines]
print()

 One-Hot Encoding dos Textos 
Matriz One-Hot Encoding de:  Investors unfazed by correction as crypto funds see $154 million inflows


,_154,_as,_by,_correction,_crypto,_funds,_inflows,_investors,_million,_see,_unfazed
0,0,0,0,0,0,0,0,1,0,0,0
1,0,0,0,0,0,0,0,0,0,0,1
2,0,0,1,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,0
5,0,0,0,0,1,0,0,0,0,0,0
6,0,0,0,0,0,1,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,1,0
8,1,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,1,0,0


Matriz One-Hot Encoding de:  Bitcoin, Ethereum prices continue descent, but crypto funds see inflows


,_bitcoin,_but,_continue,_crypto,_descent,_ethereum,_funds,_inflows,_prices,_see
0,1,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,1,0
3,0,0,1,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0
5,0,1,0,0,0,0,0,0,0,0
6,0,0,0,1,0,0,0,0,0,0
7,0,0,0,0,0,0,1,0,0,0
8,0,0,0,0,0,0,0,0,0,1
9,0,0,0,0,0,0,0,1,0,0


Matriz One-Hot Encoding de:  The surge in euro area inflation during the pandemic: transitory but with upside risks


...

,_after,_and,_bank,_bitcoin,_cryptocurrency,_cryptos,_fall,_liquidation,_news,_prices,_silvergate
0,0,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,1,0
2,0,1,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,1,0,0
4,0,0,0,1,0,0,0,0,0,0,0
5,0,0,0,0,0,1,0,0,0,0,0
6,0,0,0,0,0,0,1,0,0,0,0
7,1,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,1
9,0,0,1,0,0,0,0,0,0,0,0


Matriz One-Hot Encoding de:  Silvergate Capital To Shut Down, Liquidate Crypto-Friendly Silvergate Bank


,_bank,_capital,_cryptofriendly,_down,_liquidate,_shut,_silvergate,_to
0,0,0,0,0,0,0,1,0
1,0,1,0,0,0,0,0,0
2,0,0,0,0,0,0,0,1
3,0,0,0,0,0,1,0,0
4,0,0,0,1,0,0,0,0
5,0,0,0,0,1,0,0,0
6,0,0,1,0,0,0,0,0
7,0,0,0,0,0,0,1,0
8,1,0,0,0,0,0,0,0


Matriz One-Hot Encoding de:  Dow Jones Rises On Surprise Jump In Jobless Claims; Bitcoin Drops As Crypto Bank Silvergate Crashes 40%


,_40,_as,_bank,_bitcoin,_claims,_crashes,_crypto,_dow,_drops,_in,_jobless,_jones,_jump,_on,_rises,_silvergate,_surprise
0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
5,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
6,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
8,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
9,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


### Counting Vectors
Também conhecido como Bag of Words (BoW), é uma técnica de representação vetorial em que cada documento é representado como um vetor contendo a contagem de ocorrências das palavras no documento. É uma abordagem simples e amplamente utilizada para representar documentos em NLP.

### TF-IDF
Term Frequency-Inverse Document Frequency (TF-IDF) é uma técnica de representação vetorial que combina a frequência de termos (TF) em um documento com a frequência inversa de documentos (IDF) em um corpus. É uma técnica popular que ajuda a lidar com o desequilíbrio de frequência de palavras em diferentes documentos.

### Co-ocorrência Vectors
É uma técnica de representação vetorial que captura a frequência com que as palavras ocorrem juntas em um contexto específico. É construída com base nas co-ocorrências de palavras em documentos ou em uma janela de contexto em torno de cada palavra.

### Word2Vec
É uma técnica de representação vetorial que aprende representações densas de palavras com base em seu contexto em um grande corpus de texto. É uma abordagem de representação vetorial distribuída que tem se mostrado eficaz em capturar o significado semântico e as relações entre palavras.